In [1]:
import pandas as pd
import numpy as np

In [2]:
root = '/Volumes/Data2/RST/notebook/'

In [3]:
trial = 0
period = 'EV'

origin = 1511
dest = 1512

In [4]:
df_o = pd.read_csv(period + '/' + str(trial) + '/int-path_' + str(origin) + '_' + str(dest) + '.csv')
path_cost = pd.read_csv('AM_segment-cost.csv')

In [5]:
df_o['sequence'] = df_o.groupby('path').cumcount()

df_d = df_o.copy()

In [6]:
df_d['sequence'] = df_d['sequence'] - 1
df_d = df_d[df_d['sequence'] >= 0]

In [7]:
df_o = df_o.rename(columns = {'INT_ID':'INT_ID_o'})
df_d = df_d.rename(columns = {'INT_ID':'INT_ID_d'})

In [8]:
walk_links = pd.read_csv(root + 'GIS/int_tts_walk_time.csv')
walk_links['duration'] = round(walk_links['duration']/60).astype(int)
walk_links['gta06'] = walk_links['gta06'] + 1000
walk_links

,gta06,INT_ID,duration
0,1001,13461602,25
1,1001,13461733,21
2,1001,13462844,20
3,1001,13461863,27
4,1001,13463408,9
...,...,...,...
37157,1081,13467412,24
37158,1081,13467854,6
37159,1081,13466312,22
37160,1081,13467197,29


In [9]:
walk_o = walk_links.rename(columns = {'gta06': 'INT_ID_o', 'INT_ID':'INT_ID_d', 'duration':'cost'})
walk_d = walk_links.rename(columns = {'gta06': 'INT_ID_d', 'INT_ID':'INT_ID_o', 'duration':'cost'})

walk = walk_o.append(walk_d).copy()
walk['walk'] = True
walk

,INT_ID_o,INT_ID_d,cost,walk
0,1001,13461602,25,True
1,1001,13461733,21,True
2,1001,13462844,20,True
3,1001,13461863,27,True
4,1001,13463408,9,True
...,...,...,...,...
37157,13467412,1081,24,True
37158,13467854,1081,6,True
37159,13466312,1081,22,True
37160,13467197,1081,29,True


In [10]:
path_edges = df_o.merge(df_d).sort_values(by = ['path', 'sequence'])
path_edges

,path,INT_ID_o,sequence,INT_ID_d
0,0,1511,0,13448559
1,0,13448559,1,13448439
2,0,13448439,2,1512
3,1,1511,0,13448559
4,1,13448559,1,1512
5,2,1511,0,13449023
6,2,13449023,1,13448960
7,2,13448960,2,1512


In [11]:
cost = path_cost.drop(columns = ['std'])
cost['walk'] = False
cost = cost.append(walk)
cost

,INT_ID_o,INT_ID_d,cost,walk
0,100.0,101.0,1.321429,False
1,100.0,151.0,1.819672,False
2,100.0,13467163.0,1.000000,False
3,100.0,13467239.0,2.000000,False
4,100.0,13467593.0,3.000000,False
...,...,...,...,...
37157,13467412.0,1081.0,24.000000,True
37158,13467854.0,1081.0,6.000000,True
37159,13466312.0,1081.0,22.000000,True
37160,13467197.0,1081.0,29.000000,True


In [12]:
path_edges = path_edges.merge(cost, how = 'left')

path_edges
# edges = edges[['INT_ID_o', 'INT_ID_d', 'cost', 'type']]

,path,INT_ID_o,sequence,INT_ID_d,cost,walk
0,0,1511,0,13448559,11.0,True
1,0,13448559,1,13448439,0.0,False
2,0,13448439,2,1512,19.0,True
3,1,1511,0,13448559,11.0,True
4,1,13448559,1,1512,19.0,True
5,2,1511,0,13449023,19.0,True
6,2,13449023,1,13448960,0.0,False
7,2,13448960,2,1512,11.0,True


### Check for All-Walking

In [13]:
walk_path = path_edges.groupby('path').count()[['sequence']].copy()
walk_path['walk_links'] = path_edges.groupby('path')['walk'].sum()

walk_path_num = len(walk_path[walk_path['sequence'] == walk_path['walk_links']])

### Unique Edges

In [14]:
edges = path_edges[['INT_ID_o', 'INT_ID_d', 'cost', 'walk']].drop_duplicates().copy()

In [15]:
edges

,INT_ID_o,INT_ID_d,cost,walk
0,1511,13448559,11.0,True
1,13448559,13448439,0.0,False
2,13448439,1512,19.0,True
4,13448559,1512,19.0,True
5,1511,13449023,19.0,True
6,13449023,13448960,0.0,False
7,13448960,1512,11.0,True


### Removing Walking Links

Walking by definition has infinite redundancy

In [16]:
edges = edges[edges['walk'] == False]

### NA cost

If no cost, set it as 0.25. Also set minimum cost as 0.25

In [24]:
edges['cost'] = edges['cost'].fillna(0.5)

edges['cost'] = np.where(edges['cost'] == 0, 0.5, edges['cost'])

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_3192/2225841957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['cost'] = edges['cost'].fillna(0.5)
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_3192/2225841957.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['cost'] = np.where(edges['cost'] == 0, 0.5, edges['cost'])


In [25]:
edges

,INT_ID_o,INT_ID_d,cost,walk
1,13448559,13448439,0.5,False
6,13449023,13448960,0.5,False


In [26]:
num = 0 
denom = 0
gamma = 0

if walk_path_num > 0:
    n_routes = (path_edges['path'].max() + 1) - walk_path_num + 1
else:
    n_routes = path_edges['path'].max() + 1
    

if n_routes == 1:
    gamma = 1
else:
    for index, row in edges.iterrows():
        int_o = row['INT_ID_o']
        int_d = row['INT_ID_d']

        cost = row['cost']
        n_a = len(path_edges[(path_edges['INT_ID_o'] == int_o) & (path_edges['INT_ID_d'] == int_d)])

        iteration = (np.log(n_a) * cost) 
        denom = denom + np.log(n_routes) * cost
        num = num + iteration
        print( int_o, int_d, n_routes, n_a, cost, np.log(n_a) * cost, np.log(n_routes) * cost)

    gamma = 1 - num/denom
    gamma

13448559 13448439 3 1 0.5 0.0 0.5493061443340549
13449023 13448960 3 1 0.5 0.0 0.5493061443340549


In [31]:
if walk_path_num > 0:

    eff_paths = 1 + 1 + gamma * (n_routes - 1 - 1)

else:

    eff_paths = 1 + gamma * (n_routes - 1)
eff_paths

3.0

In [28]:
n_routes, gamma, denom, num

(3, 1.0, 1.0986122886681098, 0.0)

In [22]:
path_edges

,path,INT_ID_o,sequence,INT_ID_d,cost,walk
0,0,1511,0,13448559,11.0,True
1,0,13448559,1,13448439,0.0,False
2,0,13448439,2,1512,19.0,True
3,1,1511,0,13448559,11.0,True
4,1,13448559,1,1512,19.0,True
5,2,1511,0,13449023,19.0,True
6,2,13449023,1,13448960,0.0,False
7,2,13448960,2,1512,11.0,True


In [23]:
walk_path_num

1